In [19]:
####### Header files ##############################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2 ##For STrAdaBoost.R2
# from TwoStageTrAdaBoostR2 import TwoStageTrAdaBoostR2 ## For two-stage TrAdaBoost.R2

import pandas as pd
import sys
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Activation, Conv2D, Dropout, Flatten
from keras import optimizers, utils, initializers, regularizers
import keras.backend as K

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler #Importing the StandardScaler

from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats.stats import pearsonr
from math import sqrt

#Geo plotting libraries
#import geopandas as gdp
#from matplotlib.colors import ListedColormap
#import geoplot as glpt

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import KFold
import matplotlib.lines as mlines
import folium
import glob

from statistics import mean

pd.options.display.max_columns = None

print("Done uploading repositories")

Done uploading repositories


In [8]:
############################## UCI Italian dataset #######################################################
## 2 years and single terrain
aqi_df = pd.read_csv('AQI_datasets/UCI_AQI/AirQualityUCI.csv', sep=',', delimiter=";", decimal=",", index_col = None, header=0)

aqi_df.head(20)
# aqi_df.shape
aqi_df.columns

## Predictors: T,Ah,Rh, NMHC_GT, NOx_GT, CO_GT, C6H6_GT ,Target: O3 

Index(['Date', 'Time', 'CO_GT', 'PT08_S1_CO', 'NMHC_GT', 'C6H6_GT',
       'PT08_S2_NMHC', 'NOx_GT', 'PT08_S3_NOx', 'NO2_GT', 'PT08_S4_NO2',
       'PT08_S5_O3', 'T', 'RH', 'AH', 'Unnamed: 15', 'Unnamed: 16'],
      dtype='object')

In [9]:
def remove_outlier(col):
    aqi_df[col] = aqi_df.groupby('Date')[col].transform(lambda x: x.fillna(x.mean()))


#drop end rows with NaN values
aqi_df.dropna(how = 'all', inplace = True)

drop_unamed = ['Unnamed: 15', 'Unnamed: 16']
aqi_df = aqi_df.drop(drop_unamed, axis = 1)
aqi_df.columns
# aqi_df.shape
# aqi_df

#Observing data statistics
aqi_df.describe()

#Drop unwanted columns
drop_uw = ['Time', 'PT08_S1_CO', 'PT08_S2_NMHC', 'PT08_S5_O3', 'PT08_S3_NOx', 'PT08_S4_NO2']
aqi_df = aqi_df.drop(drop_uw, axis = 1)
aqi_df

#Replace the -200 values seen in the dataset with 
aqi_df.replace(to_replace = -200, value = np.NaN, inplace = True)

#Replace the NaN values with the column mean
col_list = aqi_df.columns[1:]
for i in col_list:
    remove_outlier(i)

aqi_df.fillna(method ='ffill', inplace= True)
aqi_df.dropna(axis = 0)
aqi_df

#Convert 'Date' column to datetime and then seperate out year and month into different columns.
aqi_df.Date = pd.to_datetime(aqi_df.Date)
aqi_df['Year'] = aqi_df['Date'].dt.year
aqi_df['Month'] = aqi_df['Date'].dt.month
drop_date = ['Date']
aqi_df = aqi_df.drop(drop_date, axis = 1)
aqi_df


,CO_GT,NMHC_GT,C6H6_GT,NOx_GT,NO2_GT,T,RH,AH,Year,Month
0,2.6,150.0,11.9,166.0,113.0,13.6,48.9,0.7578,2004,10
1,2.0,112.0,9.4,103.0,92.0,13.3,47.7,0.7255,2004,10
2,2.2,88.0,9.0,131.0,114.0,11.9,54.0,0.7502,2004,10
3,2.2,80.0,9.2,172.0,122.0,11.0,60.0,0.7867,2004,10
4,1.6,51.0,6.5,131.0,116.0,11.2,59.6,0.7888,2004,10
...,...,...,...,...,...,...,...,...,...,...
9352,3.1,275.0,13.5,472.0,190.0,21.9,29.3,0.7568,2005,4
9353,2.4,275.0,11.4,353.0,179.0,24.3,23.7,0.7119,2005,4
9354,2.4,275.0,12.4,293.0,175.0,26.9,18.3,0.6406,2005,4
9355,2.1,275.0,9.5,235.0,156.0,28.3,13.5,0.5139,2005,4


In [11]:
#Split the dataset according to the year.
aqi_df_2004 = aqi_df[aqi_df['Year'] == 2004]
aqi_df_2005 = aqi_df[aqi_df['Year'] == 2005]

aqi_df_2004 = aqi_df_2004.reset_index(drop=True)
aqi_df_2005 = aqi_df_2005.reset_index(drop=True)


#Divide the dataframe into target and the predictors.
target_uci_col = ['NOx_GT']
aqi_df_2004_target = aqi_df_2004[target_uci_col]
aqi_df_2004_target.columns = ['NOx']

aqi_df_2005_target = aqi_df_2005[target_uci_col]
aqi_df_2005_target.columns = ['NOx']

aqi_df_2004_predictors = aqi_df_2004.drop(target_uci_col, axis = 1)
aqi_df_2004_predictors.columns = ['CO', 'NMHC', 'C6H6', 'NO2', 'Temp', 'RH', 'AH', 'Year', 'Month']

aqi_df_2005_predictors = aqi_df_2005.drop(target_uci_col, axis = 1)
aqi_df_2005_predictors.columns = ['CO', 'NMHC', 'C6H6', 'NO2', 'Temp', 'RH', 'AH', 'Year', 'Month']

aqi_df_2004_target
aqi_df_2004_predictors

aqi_df_2005_target
aqi_df_2005_predictors


,CO,NMHC,C6H6,NO2,Temp,RH,AH,Year,Month
0,2.134783,275.0,4.2,128.409091,8.2,40.1,0.4375,2005,1
1,1.600000,275.0,8.8,106.000000,5.3,50.7,0.4564,2005,1
2,2.500000,275.0,7.5,129.000000,5.9,50.0,0.4689,2005,1
3,2.700000,275.0,7.6,128.409091,4.9,53.9,0.4693,2005,1
4,1.900000,275.0,5.6,126.000000,4.3,55.3,0.4650,2005,1
...,...,...,...,...,...,...,...,...,...
2242,3.100000,275.0,13.5,190.000000,21.9,29.3,0.7568,2005,4
2243,2.400000,275.0,11.4,179.000000,24.3,23.7,0.7119,2005,4
2244,2.400000,275.0,12.4,175.000000,26.9,18.3,0.6406,2005,4
2245,2.100000,275.0,9.5,156.000000,28.3,13.5,0.5139,2005,4


In [13]:
## Standardize the dataset
columns_uci = ['CO', 'NMHC', 'C6H6', 'NO2', 'Temp', 'RH', 'AH', 'Year', 'Month']
cols_to_norm = ['CO', 'NMHC', 'C6H6', 'NO2', 'Temp', 'RH', 'AH']

ss = StandardScaler()
aqi_df_2004_predictors[cols_to_norm] = ss.fit_transform(aqi_df_2004_predictors[cols_to_norm])
aqi_df_2005_predictors[cols_to_norm] = ss.fit_transform(aqi_df_2005_predictors[cols_to_norm])

aqi_df_2004_predictors
# aqi_df_2005_predictors

,CO,NMHC,C6H6,NO2,Temp,RH,AH,Year,Month
0,0.357434,-1.403072,0.167727,0.358847,-0.897718,0.030505,-1.028223,2004,10
1,-0.073637,-1.872833,-0.163670,-0.177882,-0.935168,-0.038507,-1.115824,2004,10
2,0.070053,-2.169524,-0.216693,0.384406,-1.109937,0.323803,-1.048835,2004,10
3,0.070053,-2.268421,-0.190181,0.588874,-1.222288,0.668860,-0.949843,2004,10
4,-0.361018,-2.626923,-0.548090,0.435523,-1.197321,0.645856,-0.944148,2004,10
...,...,...,...,...,...,...,...,...,...
7105,-0.648398,0.142195,-0.707160,0.052145,-1.434508,-0.849390,-2.010003,2004,12
7106,-0.648398,0.142195,-0.707160,0.052145,-1.397057,-0.924153,-2.033598,2004,12
7107,-0.648398,0.142195,-0.680649,0.052145,-1.459475,-0.912651,-2.062075,2004,12
7108,-0.648398,0.142195,-0.667393,0.052145,-1.459475,-0.642356,-1.914537,2004,12


In [14]:
#Splitting dataset into train, test and source.
X_2005_train, X_2005_test, y_2005_train, y_2005_test = train_test_split(aqi_df_2005_predictors, aqi_df_2005_target, test_size = 0.96, random_state = 1)

X_2004 = aqi_df_2004_predictors
y_2004 = aqi_df_2004_target

X_2005_train.shape


(89, 9)

In [15]:
############## Prediction for AdaBoost #########################################
predictionlist = []
r2scorelist = []
rmselist = []

print("AdaBoost")

for x in range(0, 10):

    np_TF_train_X = X_2005_train.to_numpy()
    np_TF_train_y = y_2005_train.to_numpy()

    np_test_X = X_2005_test.to_numpy()
    np_test_y = y_2005_test.to_numpy()

    np_TF_train_y_list = np_TF_train_y.ravel()
    np_test_y_list = np_test_y.ravel()

    sample_size = [len(X_2004), len(X_2005_train)]
    n_estimators = 100
    steps = 30
    fold = 10
    random_state = np.random.RandomState(1)

    regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=6), n_estimators = n_estimators)
    
    regr_2.fit(np_TF_train_X, np_TF_train_y_list)
    y_pred2 = regr_2.predict(np_test_X)
    predictionlist.append(y_pred2)
    
    rmse_adaboost = np.sqrt(mean_squared_error(np_test_y_list, y_pred2))
    print("RMSE of AdaboostR2:", rmse_adaboost)
    rmselist.append(rmse_adaboost)
    
    r2_score_adaboost_values = pearsonr(np_test_y_list, y_pred2)
    r2_score_adaboost = (r2_score_adaboost_values[0])**2
    print("R^2 of AdaboostR2:", r2_score_adaboost)
    r2scorelist.append(r2_score_adaboost)


predict_adaboost = np.mean(predictionlist, axis=0)
list_orginal_adaboost = np_test_y_list

print("mean RMSE of AdaboostR2:", mean(rmselist))
print("mean R^2 of AdaboostR2:", mean(r2scorelist))

# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.title('AdaBoost.R2 Predicted vs Actual')
# # plt.savefig("AQI_datasets/UCI_AQI_Results/NOx/AdaBoostR2_Transfer.png")
# plt.show()

with open('AQI_datasets/UCI_AQI_Results/NOx/AdaBoostR2_std.txt', 'w') as filehandle1:
    for listitem in r2scorelist:
        filehandle1.write('%s\n' % listitem)

with open('AQI_datasets/UCI_AQI_Results/NOx/AdaBoostRMSE_std.txt', 'w') as filehandle2:
    for listitem in rmselist:
        filehandle2.write('%s\n' % listitem)

with open('AQI_datasets/UCI_AQI_Results/NOx/AdaBoostprediction_std.txt', 'w') as filehandle3:
    for listitem in predict_adaboost:
        filehandle3.write('%s\n' % listitem)



AdaBoost
RMSE of AdaboostR2: 83.91320378998331
R^2 of AdaboostR2: 0.8454537219358633
RMSE of AdaboostR2: 82.04586506720729
R^2 of AdaboostR2: 0.8544853578676977
RMSE of AdaboostR2: 84.54380664086712
R^2 of AdaboostR2: 0.844849055217403
RMSE of AdaboostR2: 82.75608186761464
R^2 of AdaboostR2: 0.8535869910246799
RMSE of AdaboostR2: 84.12529629994694
R^2 of AdaboostR2: 0.8474547904979808
RMSE of AdaboostR2: 83.99132361404465
R^2 of AdaboostR2: 0.8480223121661749
RMSE of AdaboostR2: 82.91541976150522
R^2 of AdaboostR2: 0.8503802702235789
RMSE of AdaboostR2: 84.32921038035028
R^2 of AdaboostR2: 0.8431662232094902
RMSE of AdaboostR2: 82.64800807105814
R^2 of AdaboostR2: 0.8488273580046559
RMSE of AdaboostR2: 84.65070553981312
R^2 of AdaboostR2: 0.8410950280851668
mean RMSE of AdaboostR2: 83.59189210323908
mean R^2 of AdaboostR2: 0.8477321108232692


In [198]:
########################### Prediction for TrAdaBoost.R2 ##########################################
predictionlist = []
r2scorelist = []
rmselist = []

print("TrAdaBoost.R2")

for x in range(0, 10):

    TF_train_X = pd.concat([X_2004, X_2005_train], sort= False)
    TF_train_y = pd.concat([y_2004, y_2005_train], sort= False)

    np_TF_train_X = TF_train_X.to_numpy()
    np_TF_train_y = TF_train_y.to_numpy()

    np_test_X = X_2005_test.to_numpy()
    np_test_y = y_2005_test.to_numpy()

    np_TF_train_y_list = np_TF_train_y.ravel()
    np_test_y_list = np_test_y.ravel()

    sample_size = [len(X_2004), len(X_2005_train)]
    n_estimators = 100
    steps = 30
    fold = 10
    random_state = np.random.RandomState(1)

    ################################################TrAdaBoost.R2############################################################################
    regr_1 = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6), #xgb.XGBRegressor(objective ='reg:squarederror',learning_rate = 0.0001,max_depth = 6,n_estimators = 2000),
                          n_estimators = n_estimators, sample_size = sample_size,
                          steps = steps, fold = fold,
                          random_state = random_state)

    regr_1.fit(np_TF_train_X, np_TF_train_y_list)
    y_pred1 = regr_1.predict(np_test_X)
    predictionlist.append(y_pred1)

    mse_twostageboost = np.sqrt(mean_squared_error(np_test_y_list, y_pred1))
    print("RMSE of TrAdaboostR2:", mse_twostageboost)
    rmselist.append(mse_twostageboost)

    r2_score_twostageboost_values = pearsonr(np_test_y_list, y_pred1)
    r2_score_twostageboost = (r2_score_twostageboost_values[0])**2
    print("R^2 of TrAdaboostR2:", r2_score_twostageboost)
    r2scorelist.append(r2_score_twostageboost)

predict_tradaboost = np.mean(predictionlist, axis=0)
list_orginal_tradaboost = np_test_y_list

print("mean RMSE of TrAdaboostR2:", mean(rmselist))
print("mean R^2 of TrAdaboostR2:", mean(r2scorelist))

# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.title('TrAdaBoost.R2 Predicted vs Actual')
# # plt.savefig("AQI_datasets/UCI_AQI_Results/NOx/AdaBoostR2_Transfer.png")
# plt.show()

with open('AQI_datasets/UCI_AQI_Results/NOx/TrAdaBoostR2_std.txt', 'w') as filehandle1:
    for listitem in r2scorelist:
        filehandle1.write('%s\n' % listitem)

with open('AQI_datasets/UCI_AQI_Results/NOx/TrAdaBoostRMSE_std.txt', 'w') as filehandle2:
    for listitem in rmselist:
        filehandle2.write('%s\n' % listitem)

with open('AQI_datasets/UCI_AQI_Results/NOx/TrAdaBoostprediction_std.txt', 'w') as filehandle3:
    for listitem in predict_tradaboost:
        filehandle3.write('%s\n' % listitem)


TrAdaBoost.R2
RMSE of TrAdaboostR2: 171.6515413016132
R^2 of TrAdaboostR2: 0.8533748798630838
RMSE of TrAdaboostR2: 170.14551538005634
R^2 of TrAdaboostR2: 0.8533157948351648
RMSE of TrAdaboostR2: 170.69377563176536
R^2 of TrAdaboostR2: 0.8534914143838899
RMSE of TrAdaboostR2: 170.23152266767838
R^2 of TrAdaboostR2: 0.8526018221355194
RMSE of TrAdaboostR2: 172.05586024289434
R^2 of TrAdaboostR2: 0.8510481166211975
RMSE of TrAdaboostR2: 169.06705817239322
R^2 of TrAdaboostR2: 0.8554409433731467
RMSE of TrAdaboostR2: 173.20665423910825
R^2 of TrAdaboostR2: 0.848584522950516
RMSE of TrAdaboostR2: 172.54867998954228
R^2 of TrAdaboostR2: 0.8490425791881079
RMSE of TrAdaboostR2: 170.27823788650394
R^2 of TrAdaboostR2: 0.8535451402656496
RMSE of TrAdaboostR2: 170.2347615205619
R^2 of TrAdaboostR2: 0.853871346908591
mean RMSE of TrAdaboostR2: 171.01136070321172
mean R^2 of TrAdaboostR2: 0.8524316560524867


In [25]:
target_column = ['NOx']

X_train = pd.read_csv('ActiveSampling/UCI_O3_activesampling_train.csv')
X_test = pd.read_csv('ActiveSampling/UCI_O3_activesampling_test.csv')
X_source = pd.read_csv('ActiveSampling/UCI_O3_activesampling_source.csv')

print(X_train.shape)
print(X_test.shape)
print(X_source.shape)

y_2004 = X_source[target_column]
X_2004 = X_source.drop(target_column, axis = 1)

y_2005_train = X_train[target_column]
X_2005_train = X_train.drop(target_column, axis = 1)

y_2005_test = X_test[target_column]
X_2005_test = X_test.drop(target_column, axis = 1)

X_test

(98, 10)
(2158, 10)
(2000, 10)


,CO,NMHC,C6H6,NO2,Temp,RH,AH,Year,Month,NOx
0,-1.122519,0.0,-1.118436,-1.747226,-0.692434,-0.572013,-0.954190,2005,8,54.0
1,-1.122519,0.0,-1.118436,-1.252888,-1.301512,0.394126,-0.816529,2005,1,94.0
2,1.019686,0.0,1.443230,0.210353,2.120074,-0.295094,2.295396,2005,3,445.0
3,0.576471,0.0,0.806695,1.574727,0.525722,-0.178173,0.337425,2005,11,582.0
4,-0.900912,0.0,-0.761355,-0.857417,-0.423723,1.151037,0.369745,2005,3,179.0
...,...,...,...,...,...,...,...,...,...,...
2153,-0.974781,0.0,-0.947658,-0.976059,0.651121,-0.885854,-0.245937,2005,2,125.0
2154,-0.900912,0.0,-0.994234,-1.391303,1.027316,-0.313555,0.734845,2005,3,81.0
2155,2.718677,0.0,3.228634,0.724465,-0.352067,0.480278,-0.010118,2005,1,875.0
2156,-1.099435,0.0,-1.196062,-2.122923,-1.265684,0.135668,-0.926259,2005,1,42.0


In [26]:
################################# Prediction for STrAdaBoost ##################################################################
predictionlist = []
r2scorelist = []
rmselist = []

print("STrAdaBoost.R2")

for x in range(0, 10):

    TF_train_X = pd.concat([X_2004, X_2005_train], sort= False)
    TF_train_y = pd.concat([y_2004, y_2005_train], sort= False)

    np_TF_train_X = TF_train_X.to_numpy()
    np_TF_train_y = TF_train_y.to_numpy()

    np_test_X = X_2005_test.to_numpy()
    np_test_y = y_2005_test.to_numpy()

    np_TF_train_y_list = np_TF_train_y.ravel()
    np_test_y_list = np_test_y.ravel()

    sample_size = [len(X_2004), len(X_2005_train)]
    n_estimators = 100
    steps = 30
    fold = 10
    random_state = np.random.RandomState(1)

    ################################################STrAdaBoost.R2############################################################################
    regr_1 = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6), #xgb.XGBRegressor(objective ='reg:squarederror',learning_rate = 0.0001,max_depth = 6,n_estimators = 2000),
                          n_estimators = n_estimators, sample_size = sample_size,
                          steps = steps, fold = fold,
                          random_state = random_state)

    regr_1.fit(np_TF_train_X, np_TF_train_y_list)
    y_pred1 = regr_1.predict(np_test_X)
    predictionlist.append(y_pred1)

    mse_twostageboost = np.sqrt(mean_squared_error(np_test_y_list, y_pred1))
    print("RMSE of STrAdaboostR2:", mse_twostageboost)
    rmselist.append(mse_twostageboost)

    r2_score_twostageboost_values = pearsonr(np_test_y_list, y_pred1)
    r2_score_twostageboost = (r2_score_twostageboost_values[0])**2
    print("R^2 of STrAdaboostR2:", r2_score_twostageboost)
    r2scorelist.append(r2_score_twostageboost)

predict_stradaboost = np.mean(predictionlist, axis=0)
list_orginal_stradaboost = np_test_y_list

print("mean RMSE of STrAdaboostR2:", mean(rmselist))
print("mean R^2 of STrAdaboostR2:", mean(r2scorelist))


with open('AQI_datasets/UCI_AQI_Results/NOx/STrAdaBoostR2_std.txt', 'w') as filehandle1:
    for listitem in r2scorelist:
        filehandle1.write('%s\n' % listitem)

with open('AQI_datasets/UCI_AQI_Results/NOx/STrAdaBoostRMSE_std.txt', 'w') as filehandle2:
    for listitem in rmselist:
        filehandle2.write('%s\n' % listitem)

with open('AQI_datasets/UCI_AQI_Results/NOx/STrAdaBoostprediction_std.txt', 'w') as filehandle3:
    for listitem in predict_stradaboost:
        filehandle3.write('%s\n' % listitem)



plt.scatter(predict_adaboost, list_orginal_adaboost,  c ="red", alpha=0.6)
plt.scatter(predict_tradaboost, list_orginal_tradaboost,  c ="blue", alpha=0.4)
plt.scatter(predict_stradaboost, list_orginal_stradaboost,  c ="green", alpha=0.2)

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Predicted vs Actual')
# plt.savefig("AQI_datasets/UCI_AQI_Results/NOx/AdaBoostR2_Transfer.png")
plt.show()


############## How to superimpose scatter plots ########################
## https://stackoverflow.com/questions/11190735/python-matplotlib-superimpose-scatter-plots

STrAdaBoost.R2
Inside STrAdaBoost.R2
RMSE of STrAdaboostR2: 91.98994707864414
R^2 of STrAdaboostR2: 0.82613619429586
Inside STrAdaBoost.R2


KeyboardInterrupt: 